## Step 30: Build Rainfall_Distribution Coverage

In [1]:
import arcpy,os,zipfile,requests,shutil,json,http.client;

# Set Conneticut RF values by county
ct_rf_values = {
     'Fairfield' : 'NRCC_C'
    ,'Litchfield': 'NRCC_C'
    ,'New Haven' : 'NRCC_D'
    ,'Hartford'  : 'NRCC_C'
    ,'Middlesex' : 'NRCC_C'
    ,'Tolland'   : 'NRCC_C'
    ,'New London': 'NRCC_C'
    ,'Windham'   : 'NRCC_C'
}

# Verify or create source filegeodatabase
fgdb = os.getcwd() + os.sep + 'source.gdb';

if not arcpy.Exists(fgdb):
   arcpy.CreateFileGDB_management(
       os.path.dirname(fgdb)
      ,os.path.basename(fgdb)
   );

# Verify or create results filegeodatabase
rezfgdb = os.getcwd() + os.sep + 'results.gdb';

if not arcpy.Exists(rezfgdb):
   arcpy.CreateFileGDB_management(
       os.path.dirname(rezfgdb)
      ,os.path.basename(rezfgdb)
   );

arcpy.Exists(fgdb)

True

### Define the source datasets to fetch

In [2]:
data = {};

data['ireg'] = {
     'file' : 'files/ireg_zones.zip'
    ,'shp'  : 'ireg_zones'
};

data['ohiovalley'] = {
     'file' : 'files/VA_GIS.zip'
    ,'zipdr': 'VA_zipped'
    ,'shp'  : 'NOAA_14_east_rf_dist'
};

data['al'] = {
     'file' : 'files/NOAA_14_AL.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_AL.zip'
    ,'zipdr': 'NOAA_14_AL'
    ,'shp'  : 'AL_county_split_rf_dist__WGS'
};

data['ar'] = {
     'file' : 'files/NOAA_14_AR.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_AR.zip'
    ,'zipdr': 'NOAA_14_AR'
    ,'shp'  : 'AR_county_split_WGS'
};

data['ca'] = {
     'file' : 'files/NOAA_14_CA.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_CA.zip'
    ,'zipdr': 'NOAA_14_CA'
    ,'shp'  : 'CA_6_dist_map'
};
data['co'] = {
     'file' : 'files/NOAA_14_CO.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_CO.zip'
    ,'zipdr': 'NOAA_14_CO'
    ,'shp'  : 'CO_NOAA_14_rf_dist'
};

data['de'] = {
     'file' : 'files/NOAA_14_DE.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_DE.zip'
    ,'zipdr': 'NOAA_14_DE'
    ,'shp'  : 'DE_counties_WGS72'
};

data['fl'] = {
     'file' : 'files/NOAA_14_FL_GIS.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_FL_GIS.zip'
    ,'shp'  : 'county_FL_WGS_split_poly9'
};

data['flpt'] = {
     'file' : 'files/NOAA_14_FL_GIS.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_FL_GIS.zip'
    ,'shp'  : 'FL_NOAA_14_pts'
};

data['ga'] = {
     'file' : 'files/NOAA_14_GA.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_GA.zip'
    ,'zipdr': 'NOAA_14_GA'
    ,'shp'  : 'GA_county_split_WGS'
    ,'query': "RF_DIST IS NOT NULL AND RF_DIST > 0 "
};

data['ia'] = {
     'file' : 'files/NOAA_14_IA.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_IA.zip'
    ,'zipdr': 'NOAA_14_IA'
    ,'shp'  : 'county_IA_WGS'
};

data['in'] = {
     'file' : 'files/NOAA_14_IN.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_IN.zip'
    ,'zipdr': 'NOAA_14_IN'
    ,'shp'  : 'IN_counties_WGS'
};

data['ky'] = {
     'file' : 'files/NOAA_14_KY.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_KY.zip'
    ,'zipdr': 'NOAA_14_KY'
    ,'shp'  : 'KY_counties_WGS'
};

data['la'] = {
     'file' : 'files/NOAA_14_LA.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_LA.zip'
    ,'zipdr': 'NOAA_14_LA'
    ,'shp'  : 'LA_Parish_split_poly'
};

data['md'] = {
     'file' : 'files/NOAA_14_MD.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_MD.zip'
    ,'zipdr': 'NOAA_14_MD'
    ,'shp'  : 'MD_areas_WGS72'
    ,'query': "RF_Dist IS NOT NULL AND RF_Dist <> ' ' "
};

data['mn'] = {
     'file' : 'files/NOAA_14_MN.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_MN.zip'
    ,'zipdr': 'NOAA_14_MN'
    ,'shp'  : 'county_MN_WGS'
};

data['ne'] = {
     'file' : 'files/NOAA_14_NE.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_NE.zip'
    ,'zipdr': 'NOAA_14_NE'
    ,'shp'  : 'county_NE_WGS'
};

data['nj'] = {
     'file' : 'files/NOAA_14_NJ.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_NJ.zip'
    ,'zipdr': 'NOAA_14_NJ'
    ,'shp'  : 'NJ_county_WGS72'
};

data['nv'] = {
     'file' : 'files/NOAA_14_NV.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_NV.zip'
    ,'zipdr': 'NOAA_14_NV'
    ,'shp'  : 'NV_County_WGS'
    ,'query': "State = 'NV' "
};

data['oh'] = {
     'file' : 'files/NOAA_14_OH.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_OH.zip'
    ,'zipdr': 'NOAA_14_OH'
    ,'shp'  : 'OH_counties_WGS72'
};

data['pa'] = {
     'file' : 'files/NOAA_14_PA.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_PA.zip'
    ,'zipdr': 'NOAA_14_PA'
    ,'shp'  : 'PA_counties_WGS72'
};

data['sc'] = {
     'file' : 'files/NOAA_14_SC.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_SC.zip'
    ,'zipdr': 'NOAA_14_SC'
    ,'shp'  : 'SC_NOAA_14_rf_dist'
};

data['tn'] = {
     'file' : 'files/NOAA_14_TN.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_TN.zip'
    ,'zipdr': 'NOAA_14_TN'
    ,'shp'  : 'TN_NOAA_14_rf_dist'
};

data['va'] = {
     'file' : 'files/NOAA_14_VA.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_VA.zip'
    ,'zipdr': 'NOAA_14_VA'
    ,'shp'  : 'VA_counties _and_zones'
    ,'fix'  : 'VA_counties_and_zones'
};

data['wv'] = {
     'file' : 'files/NOAA_14_WV.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NOAA/NOAA_14_WV.zip'
    ,'zipdr': 'NOAA_14_WV'
    ,'shp'  : 'WV_rainfall_distribution'
};

data['ct'] = {
     'file' : 'files/NRCC_CT.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NRCC/NRCC_CT.zip'
    ,'zipdr': 'NRCC_CT'
    ,'shp'  : 'CT_counties_WGS72'
};

data['ma'] = {
     'file' : 'files/NRCC_MA.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NRCC/NRCC_MA.zip'
    ,'zipdr': 'NRCC_MA'
    ,'shp'  : 'MA_county_WGS72'
    ,'query': "RF_Dist IS NOT NULL AND RF_Dist <> ' ' "
};

data['me'] = {
     'file' : 'files/NRCC_ME.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NRCC/NRCC_ME.zip'
    ,'zipdr': 'NRCC_ME'
    ,'shp'  : 'ME_counties_WGS72'
};

data['nh'] = {
     'file' : 'files/NRCC_NH.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NRCC/NRCC_NH.zip'
    ,'zipdr': 'NRCC_NH'
    ,'shp'  : 'NH_towns_WGS'
};

data['ny'] = {
     'file' : 'files/NRCC_NY.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NRCC/NRCC_NY.zip'
    ,'zipdr': 'NRCC_NY'
    ,'shp'  : 'NY_counties_WGS72'
    ,'query': "Rf_Dist IS NOT NULL AND Rf_Dist <> ' ' "
};

data['ri'] = {
     'file' : 'files/NRCC_RI.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NRCC/NRCC_RI.zip'
    ,'zipdr': 'NRCC_RI'
    ,'shp'  : 'RI_counties_WGS72'
};

data['vt'] = {
     'file' : 'files/NRCC_VT.zip'
    ,'url'  : 'http://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/WinTR20/NRCC/NRCC_VT.zip'
    ,'zipdr': 'NRCC_VT'
    ,'shp'  : 'VT_counties_WGS72'
};


### Download and/or extract source zip files

In [3]:
%%time

def fetchResource(data,fgdb):
    
    if 'zipdr' in data:
        zipshp = data['zipdr'] + '/' + data['shp'];
    else:
        zipshp = data['shp'];
        
    if 'query' in data:
        query = data['query'];
    else:
        query = None;
        
    if 'fix' in data:
        fix = data['fix'];
    else:
        fix = data['shp'];
    
    file = os.getcwd() + os.sep + data['file'];

    if not os.path.exists(file):
        
        r = requests.get(data['url'],stream=True);
        
        with open(file,'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk:
                    f.write(chunk);
    
    if arcpy.Exists(arcpy.env.scratchFolder + os.sep + data['shp'] + '.shp'):
        arcpy.Delete_management(arcpy.env.scratchFolder + os.sep + data['shp'] + '.shp');
        
    with zipfile.ZipFile(file) as zf:
        
        for ext in ['.dbf','.prj','sbn','.sbx','.shp','.shx']:
            if zipshp + ext in zf.namelist():
                with zf.open(zipshp + ext) as s:
                    with open(arcpy.env.scratchFolder + os.sep + fix + ext,'wb') as t:
                        shutil.copyfileobj(s,t);
        
    if arcpy.Exists(fgdb + os.sep + fix):
        arcpy.Delete_management(fgdb + os.sep + fix);

    print("  processing " + fix);
    arcpy.conversion.FeatureClassToFeatureClass(
         in_features  = arcpy.env.scratchFolder + os.sep + fix + '.shp'
        ,out_path     = fgdb
        ,out_name     = fix
        ,where_clause = query
    );

for key,val in data.items():
    
    fetchResource(val,fgdb);


  processing ireg_zones
  processing NOAA_14_east_rf_dist
  processing AL_county_split_rf_dist__WGS
  processing AR_county_split_WGS
  processing CA_6_dist_map
  processing CO_NOAA_14_rf_dist
  processing DE_counties_WGS72
  processing county_FL_WGS_split_poly9
  processing FL_NOAA_14_pts
  processing GA_county_split_WGS
  processing county_IA_WGS
  processing IN_counties_WGS
  processing KY_counties_WGS
  processing LA_Parish_split_poly
  processing MD_areas_WGS72
  processing county_MN_WGS
  processing county_NE_WGS
  processing NJ_county_WGS72
  processing NV_County_WGS
  processing OH_counties_WGS72
  processing PA_counties_WGS72
  processing SC_NOAA_14_rf_dist
  processing TN_NOAA_14_rf_dist
  processing VA_counties_and_zones
  processing WV_rainfall_distribution
  processing CT_counties_WGS72
  processing MA_county_WGS72
  processing ME_counties_WGS72
  processing NH_towns_WGS
  processing NY_counties_WGS72
  processing RI_counties_WGS72
  processing VT_counties_WGS72
Wall time: 

In [4]:
def scrape_ags(host,path,fgdb,fc,forcelimit=None):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    headers = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain"};
    conn = http.client.HTTPSConnection(host);
    conn.request("POST",path,"f=json",headers);
    response = conn.getresponse();
    data = response.read();
    json_data = json.loads(data);
    if not 'currentVersion' in json_data:
        raise ValueError("Error, unable to query https://" + host + path);
    extraction_amount = json_data['maxRecordCount'];
    if forcelimit is not None and forcelimit < extraction_amount:
        extraction_amount = forcelimit;
    where = "1=1";
    params = "where={}&returnIdsOnly=true&returnGeometry=false&f=json".format(where);
    conn = http.client.HTTPSConnection(host);
    conn.request("POST",path + "/query",params,headers);
    response = conn.getresponse();
    data = response.read();
    json_data = json.loads(data);
    ary_oid   = sorted(json_data['objectIds']);
    oid_name  = json_data['objectIdFieldName'];
    oid_count = len(ary_oid);
    initial_hit = True;
    counter = 0;
    while counter <= oid_count - 1:
        if counter + extraction_amount > oid_count - 1:
            int_max = oid_count - 1;
        else:
            int_max = counter + extraction_amount - 1;
        where = oid_name + ' >= ' + str(ary_oid[counter]) + ' AND ' + oid_name + ' <= ' + str(ary_oid[int_max]);
        print("  pulling records where " + where);
        fields = "*";
        params = "where={}&outFields={}&returnGeometry=true&f=json".format(where, fields);
        conn = http.client.HTTPSConnection(host);
        conn.request("POST",path + "/query",params,headers);
        response = conn.getresponse();
        data = response.read(); 
        json_data = json.loads(data);
        ef = arcpy.AsShape(json_data,True);
        if initial_hit:
            arcpy.management.CopyFeatures(ef,fgdb + os.sep + fc)
            initial_hit = False;
        else:
            arcpy.Append_management(ef,fgdb + os.sep + fc,"NO_TEST");
        counter += extraction_amount;
    conn.close(); 
    del conn;
    print("  Scrape complete.");
    return True;


### Download Tiger 2020 State Coverage

In [5]:
%%time

host = "tigerweb.geo.census.gov";
path = "/arcgis/rest/services/TIGERweb/State_County/MapServer/0";
fc   = "census_states";

if arcpy.Exists(fgdb + os.sep + fc):
    arcpy.Delete_management(fgdb + os.sep + fc);

z = scrape_ags(host,path,fgdb,fc,5);

z = arcpy.management.AddIndex(
     in_table   = fgdb + os.sep + fc
    ,fields     = 'GEOID'
    ,index_name = 'GEOID_IDX'
);


  pulling records where OBJECTID >= 1 AND OBJECTID <= 5
  pulling records where OBJECTID >= 6 AND OBJECTID <= 10
  pulling records where OBJECTID >= 11 AND OBJECTID <= 15
  pulling records where OBJECTID >= 16 AND OBJECTID <= 20
  pulling records where OBJECTID >= 21 AND OBJECTID <= 25
  pulling records where OBJECTID >= 26 AND OBJECTID <= 30
  pulling records where OBJECTID >= 31 AND OBJECTID <= 35
  pulling records where OBJECTID >= 36 AND OBJECTID <= 40
  pulling records where OBJECTID >= 41 AND OBJECTID <= 45
  pulling records where OBJECTID >= 46 AND OBJECTID <= 50
  pulling records where OBJECTID >= 51 AND OBJECTID <= 55
  pulling records where OBJECTID >= 56 AND OBJECTID <= 56
  Scrape complete.
Wall time: 1min 20s


In [6]:
wrkfgdb = os.getcwd() + os.sep + 'working.gdb';

if not arcpy.Exists(wrkfgdb):
   arcpy.CreateFileGDB_management(
       os.path.dirname(wrkfgdb)
      ,os.path.basename(wrkfgdb)
   );

arcpy.Exists(wrkfgdb)

True

### Fix problems with Florida County layer lacking RF information

In [7]:
%%time

wrkFL  = wrkfgdb + os.sep + 'county_FL_WGS_split_poly9_FIX';
wrkFL2 = wrkfgdb + os.sep + 'county_FL_WGS_split_poly9_NAD83';
wrkFL3 = wrkfgdb + os.sep + 'county_FL_WGS_split_poly9_RDY';

if arcpy.Exists(wrkFL):
    arcpy.Delete_management(wrkFL);

print("  spatial join FL counties with point data");
arcpy.analysis.SpatialJoin(
     target_features   = fgdb + os.sep + 'county_FL_WGS_split_poly9'
    ,join_features     = fgdb + os.sep + 'FL_NOAA_14_pts'
    ,out_feature_class = wrkFL
    ,join_operation    = 'JOIN_ONE_TO_ONE'
    ,join_type         = 'KEEP_ALL'
    ,match_option      = 'INTERSECT'
);

flds = [
     'CNTYSHORT'
    ,'COUNTYNAME'
    ,'rf_dist'
];

print("  load matches into reference hash");
cntyhash = {};
cnty2hash = {};
with arcpy.da.SearchCursor(wrkFL,flds,'rf_dist IS NOT NULL') as incur:
    for row in incur:
        if row[0] != ' ':
            cntyhash[row[0]]  = row[2];
        if row[1] != ' ':
            cnty2hash[row[1]] = row[2];
        
print("  use hash to populate other county portions and remove small dross");
with arcpy.da.UpdateCursor(wrkFL,flds,'rf_dist IS NULL') as upcur:
    for row in upcur:
        
        if row[0] in cntyhash:
            row[2] = cntyhash[row[0]];
            upcur.updateRow(row);
            
        else:
            if row[1] in cnty2hash:
                row[2] = cnty2hash[row[1]];
                upcur.updateRow(row);
                
            else:
                upcur.deleteRow();

if arcpy.Exists(wrkFL2):
    arcpy.Delete_management(wrkFL2);
    
print("  project results into NAD83")
arcpy.management.Project(
     in_dataset      = wrkFL
    ,out_dataset     = wrkFL2
    ,out_coor_system = arcpy.SpatialReference(4269)
);

if arcpy.Exists(wrkFL3):
    arcpy.Delete_management(wrkFL3);

print("  create ready version of Florida data");
arcpy.CreateFeatureclass_management(
     out_path      = wrkfgdb
    ,out_name      = "county_FL_WGS_split_poly9_RDY"
    ,geometry_type = "POLYGON"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

arcpy.management.AddFields(
     in_table          = wrkFL3
    ,field_description = [
         ['state'     ,'TEXT','state'     ,2]
        ,['rf_value'  ,'TEXT','rf_value'  ,255]
        ,['source_rf' ,'TEXT','source_rf' ,255]
        ,['source_geo','TEXT','source_geo',255]
    ]
);

fldsout = [
     'state'
    ,'rf_value'
    ,'source_rf'
    ,'source_geo'
    ,'SHAPE@'
];

fldsin = [
     'rf_dist'
    ,'SHAPE@'
];

with arcpy.da.InsertCursor(wrkFL3,fldsout) as outcur:

    with arcpy.da.SearchCursor(wrkFL2,fldsin) as incur:
        for row in incur:
            outcur.insertRow((
                 'FL'
                ,'MSE_' + row[0]
                ,'FL_NOAA_14_pts'
                ,'county_FL_WGS_split_poly9'
                ,row[1]
            ));


  spatial join FL counties with point data
  load matches into reference hash
  use hash to populate other county portions and remove small dross
  project results into NAD83
  create ready version of Florida data
Wall time: 9.47 s


### Clip out Illinois data from Ohio Valley rainfall map

In [8]:
%%time

wrkIL  = wrkfgdb + os.sep + 'rainfall_IL_NAD83';
wrkIL2 = wrkfgdb + os.sep + 'rainfall_IL_CLIP';
wrkIL3 = wrkfgdb + os.sep + 'rainfall_IL_RDY';

if arcpy.Exists(wrkIL):
    arcpy.Delete_management(wrkIL);
    
if arcpy.Exists('memory/illinois'):
    arcpy.Delete_management('memory/illinois');

print("  load state census outline into memory");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features  = fgdb + os.sep + 'census_states'
    ,out_path     = 'memory'
    ,out_name     = 'illinois'
    ,where_clause = "GEOID = '17' "
);

print("  project NOAA_14_east_rf_dist into NAD83");
arcpy.management.Project(
     in_dataset      = fgdb + os.sep + 'NOAA_14_east_rf_dist'
    ,out_dataset     = wrkIL
    ,out_coor_system = arcpy.SpatialReference(4269)
);

if arcpy.Exists(wrkIL2):
    arcpy.Delete_management(wrkIL2);

print("  clip out Illinois portion of NOAA_14_east_rf_dist");
arcpy.analysis.Clip(
     in_features       = wrkIL
    ,clip_features     = 'memory/illinois'
    ,out_feature_class = wrkIL2
);

if arcpy.Exists(wrkIL3):
    arcpy.Delete_management(wrkIL3);

print("  create ready version of Illinois data");
arcpy.CreateFeatureclass_management(
     out_path      = wrkfgdb
    ,out_name      = 'rainfall_IL_RDY'
    ,geometry_type = "POLYGON"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

arcpy.management.AddFields(
     in_table          = wrkIL3
    ,field_description = [
         ['state'     ,'TEXT','state'     ,2]
        ,['rf_value'  ,'TEXT','rf_value'  ,255]
        ,['source_rf' ,'TEXT','source_rf' ,255]
        ,['source_geo','TEXT','source_geo',255]
    ]
);

fldsout = [
     'state'
    ,'rf_value'
    ,'source_rf'
    ,'source_geo'
    ,'SHAPE@'
];

fldsin = [
     'RF_Dist'
    ,'SHAPE@'
];

with arcpy.da.InsertCursor(wrkIL3,fldsout) as outcur:

    with arcpy.da.SearchCursor(wrkIL2,fldsin) as incur:
        for row in incur:
            outcur.insertRow((
                 'IL'
                ,'NOAA_' + row[0]
                ,'NOAA_14_east_rf_dist'
                ,'NOAA_14_east_rf_dist'
                ,row[1]
            ));


  load state census outline into memory
  project NOAA_14_east_rf_dist into NAD83
  clip out Illinois portion of NOAA_14_east_rf_dist
  create ready version of Illinois data
Wall time: 7.74 s


### Clip out North Carolina portion of Ohio Valley rainfall map

In [9]:
%%time

wrkNC  = wrkfgdb + os.sep + 'rainfall_NC_NAD83';
wrkNC2 = wrkfgdb + os.sep + 'rainfall_NC_CLIP';
wrkNC3 = wrkfgdb + os.sep + 'rainfall_NC_RDY';

if arcpy.Exists(wrkNC):
    arcpy.Delete_management(wrkNC);
    
if arcpy.Exists('memory/northcarolina'):
    arcpy.Delete_management('memory/northcarolina');

print("  load state census outline into memory");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features  = fgdb + os.sep + 'census_states'
    ,out_path     = 'memory'
    ,out_name     = 'northcarolina'
    ,where_clause = "GEOID = '37' "
);

print("  project NOAA_14_east_rf_dist into NAD83");
arcpy.management.Project(
     in_dataset      = fgdb + os.sep + 'NOAA_14_east_rf_dist'
    ,out_dataset     = wrkNC
    ,out_coor_system = arcpy.SpatialReference(4269)
);

if arcpy.Exists(wrkNC2):
    arcpy.Delete_management(wrkNC2);

print("  clip out North Carolina portion of NOAA_14_east_rf_dist");
arcpy.analysis.Clip(
     in_features       = wrkNC
    ,clip_features     = 'memory/northcarolina'
    ,out_feature_class = wrkNC2
);

if arcpy.Exists(wrkNC3):
    arcpy.Delete_management(wrkNC3);

print("  create ready version of North Carolina data");
arcpy.CreateFeatureclass_management(
     out_path      = wrkfgdb
    ,out_name      = 'rainfall_NC_RDY'
    ,geometry_type = "POLYGON"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

arcpy.management.AddFields(
     in_table          = wrkNC3
    ,field_description = [
         ['state'     ,'TEXT','state'     ,2]
        ,['rf_value'  ,'TEXT','rf_value'  ,255]
        ,['source_rf' ,'TEXT','source_rf' ,255]
        ,['source_geo','TEXT','source_geo',255]
    ]
);

fldsout = [
     'state'
    ,'rf_value'
    ,'source_rf'
    ,'source_geo'
    ,'SHAPE@'
];

fldsin = [
     'RF_Dist'
    ,'SHAPE@'
];

with arcpy.da.InsertCursor(wrkNC3,fldsout) as outcur:

    with arcpy.da.SearchCursor(wrkNC2,fldsin) as incur:
        for row in incur:
            outcur.insertRow((
                 'NC'
                ,'NOAA_' + row[0]
                ,'NOAA_14_east_rf_dist'
                ,'NOAA_14_east_rf_dist'
                ,row[1]
            ));


  load state census outline into memory
  project NOAA_14_east_rf_dist into NAD83
  clip out North Carolina portion of NOAA_14_east_rf_dist
  create ready version of North Carolina data
Wall time: 7.33 s


### Replace Virginia Undefined federal lands with Ohio Valley rainfall map

In [10]:
%%time

wrkOhio = wrkfgdb + os.sep + 'NOAA_14_east_rf_dist_NAD83';
wrkVA   = wrkfgdb + os.sep + 'VA_counties_and_zones_NAD83';
wrkVA2  = wrkfgdb + os.sep + 'VA_counties_and_zones_CLIP';
wrkVA3  = wrkfgdb + os.sep + 'VA_counties_and_zones_RDY';

if arcpy.Exists(wrkOhio):
    arcpy.Delete_management(wrkOhio);

print("  project NOAA_14_east_rf_dist into NAD83");
arcpy.management.Project(
     in_dataset      = fgdb + os.sep + 'NOAA_14_east_rf_dist'
    ,out_dataset     = wrkOhio
    ,out_coor_system = arcpy.SpatialReference(4269)
);

if arcpy.Exists(wrkVA):
    arcpy.Delete_management(wrkVA);

print("  project VA_counties_and_zones into NAD83");
arcpy.management.Project(
     in_dataset      = fgdb + os.sep + 'VA_counties_and_zones'
    ,out_dataset     = wrkVA
    ,out_coor_system = arcpy.SpatialReference(4269)
);

if arcpy.Exists('memory/va_rainfall'):
    arcpy.Delete_management('memory/va_rainfall');
    
if arcpy.Exists('memory/va_federal_lands'):
    arcpy.Delete_management('memory/va_federal_lands');

print("  select Virginia zones with rainfall values defined into memory");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features  = wrkVA
    ,out_path     = 'memory'
    ,out_name     = 'va_rainfall'
    ,where_clause = "RF_Dist <> 'Undefined' "
);
  
print("  select Virginia zones with undefined rainfall values into memory");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features  = wrkVA
    ,out_path     = 'memory'
    ,out_name     = 'va_federal_lands'
    ,where_clause = "RF_Dist = 'Undefined' "
);

if arcpy.Exists(wrkVA2):
    arcpy.Delete_management(wrkVA2);

print("  clip the NOAA_14_east_rf_dist layer using the zones with undefined rainfall values")
arcpy.analysis.Clip(
     in_features       = wrkOhio
    ,clip_features     = 'memory/va_federal_lands'
    ,out_feature_class = wrkVA2
);

if arcpy.Exists(wrkVA3):
    arcpy.Delete_management(wrkVA3);

print("  add back together the Virginia zones with defined rainfall and clipped NOAA_14_rf_dist as ready layer");
arcpy.CreateFeatureclass_management(
     out_path      = wrkfgdb
    ,out_name      = 'VA_counties_and_zones_RDY'
    ,geometry_type = "POLYGON"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

arcpy.management.AddFields(
     in_table          = wrkVA3
    ,field_description = [
         ['state'     ,'TEXT','state'     ,2]
        ,['rf_value'  ,'TEXT','rf_value'  ,255]
        ,['source_rf' ,'TEXT','source_rf' ,255]
        ,['source_geo','TEXT','source_geo',255]
    ]
);

fldsout = [
     'state'
    ,'rf_value'
    ,'source_rf'
    ,'source_geo'
    ,'SHAPE@'
];

with arcpy.da.InsertCursor(wrkVA3,fldsout) as outcur:
    
    fldsin = [
         'RF_Dist'
        ,'SHAPE@'
    ];

    with arcpy.da.SearchCursor('memory/va_rainfall',fldsin) as incur:
        for row in incur:
            outcur.insertRow((
                 'VA'
                ,'NOAA_' + row[0]
                ,'VA_counties_and_zones'
                ,'VA_counties_and_zones'
                ,row[1]
            ));
            
    fldsin = [
         'RF_Dist'
        ,'SHAPE@'
    ];

    with arcpy.da.SearchCursor(wrkVA2,fldsin) as incur:
        for row in incur:
            outcur.insertRow((
                 'VA'
                ,'NOAA_' + row[0]
                ,'NOAA_14_east_rf_dist'
                ,'VA_counties_and_zones'
                ,row[1]
            ));


  project NOAA_14_east_rf_dist into NAD83
  project VA_counties_and_zones into NAD83
  select Virginia zones with rainfall values defined into memory
  select Virginia zones with undefined rainfall values into memory
  clip the NOAA_14_east_rf_dist layer using the zones with undefined rainfall values
  add back together the Virginia zones with defined rainfall and clipped NOAA_14_rf_dist as ready layer
Wall time: 10.2 s


### Alter all of Alaska to proper prefix

In [11]:
%%time

wrkAK   = wrkfgdb + os.sep + 'rainfall_AK_NAD83';
wrkAK2  = wrkfgdb + os.sep + 'rainfall_AK_RDY';

if arcpy.Exists(wrkAK):
    arcpy.Delete_management(wrkAK);
    
print("  load state census outline into memory");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features  = fgdb + os.sep + 'census_states'
    ,out_path     = wrkfgdb
    ,out_name     = 'rainfall_AK_NAD83'
    ,where_clause = "GEOID = '02' "
);

if arcpy.Exists(wrkAK2):
    arcpy.Delete_management(wrkAK2);

print("  use outline to define Alaska as SCS_I");
arcpy.CreateFeatureclass_management(
     out_path      = wrkfgdb
    ,out_name      = 'rainfall_AK_RDY'
    ,geometry_type = "POLYGON"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

arcpy.management.AddFields(
     in_table          = wrkAK2
    ,field_description = [
         ['state'     ,'TEXT','state'     ,2]
        ,['rf_value'  ,'TEXT','rf_value'  ,255]
        ,['source_rf' ,'TEXT','source_rf' ,255]
        ,['source_geo','TEXT','source_geo',255]
    ]
);

fldsout = [
     'state'
    ,'rf_value'
    ,'source_rf'
    ,'source_geo'
    ,'SHAPE@'
];

fldsin = [
    'SHAPE@'
];

with arcpy.da.InsertCursor(wrkAK2,fldsout) as outcur:

    with arcpy.da.SearchCursor(wrkAK,fldsin) as incur:
        for row in incur:
            outcur.insertRow((
                 'AK'
                ,'SCS_I' 
                ,'NOAA Documented'
                ,'Census Tiger States'
                ,row[0]
            ));


  load state census outline into memory
  use outline to define Alaska as SCS_I
Wall time: 4.62 s


### Process Conneticut with custom handling of C and D values

In [12]:
%%time

wrkCT   = wrkfgdb + os.sep + 'CT_counties_WGS72_NAD83';
wrkCT2  = wrkfgdb + os.sep + 'CT_counties_WGS72_RDY';

if arcpy.Exists(wrkCT):
    arcpy.Delete_management(wrkCT);
    
print("  project CT_counties_WGS72 into NAD83");
arcpy.management.Project(
     in_dataset      = fgdb + os.sep + 'CT_counties_WGS72'
    ,out_dataset     = wrkCT
    ,out_coor_system = arcpy.SpatialReference(4269)
);

print("  extract county name values and their centroid numbers into a reference hash");
fldsin = [
     'Name'
    ,'CENT_LONG'
    ,'CENT_LAT'
];

centroid_hash = {};
with arcpy.da.SearchCursor(wrkCT,fldsin,"Name is not NULL AND Name <> ' ' ") as incur:
    for row in incur:
        centroid_hash[str(row[1]) + ':' + str(row[2])] = row[0];
        
print("  apply hashed county names onto unnamed polygons");
with arcpy.da.UpdateCursor(wrkCT,fldsin,"Name IS NULL OR Name = ' ' ") as upcur:
    for row in upcur:
        row[0] = centroid_hash[str(row[1]) + ':' + str(row[2])];
        upcur.updateRow(row);

if arcpy.Exists(wrkCT2):
    arcpy.Delete_management(wrkCT2);
    
print("  update rf values of all CT polygons from rf_values hash")
arcpy.CreateFeatureclass_management(
     out_path      = wrkfgdb
    ,out_name      = 'CT_counties_WGS72_RDY'
    ,geometry_type = "POLYGON"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

arcpy.management.AddFields(
     in_table          = wrkCT2
    ,field_description = [
         ['state'     ,'TEXT','state'     ,2]
        ,['rf_value'  ,'TEXT','rf_value'  ,255]
        ,['source_rf' ,'TEXT','source_rf' ,255]
        ,['source_geo','TEXT','source_geo',255]
    ]
);

fldsout = [
     'state'
    ,'rf_value'
    ,'source_rf'
    ,'source_geo'
    ,'SHAPE@'
];

fldsin = [
     'Name'
    ,'SHAPE@'
];

with arcpy.da.InsertCursor(wrkCT2,fldsout) as outcur:

    with arcpy.da.SearchCursor(wrkCT,fldsin) as incur:
        for row in incur:
            outcur.insertRow((
                 'CT'
                ,ct_rf_values[row[0]]
                ,'SWC allocation'
                ,'CT_counties_WGS72'
                ,row[1]
            ));


  project CT_counties_WGS72 into NAD83
  extract county name values and their centroid numbers into a reference hash
  apply hashed county names onto unnamed polygons
  update rf values of all CT polygons from rf_values hash
Wall time: 5.46 s


### Reproject to NAD83 and reformat to common layout for loading.

In [13]:
%%time

layers = [
     ['AL','AL_county_split_rf_dist__WGS','RF_DIST','MSE']
    ,['AR','AR_county_split_WGS'         ,'RF_Dist','MSE']
    ,['CA','CA_6_dist_map'               ,'RF_dist','CA']
    ,['CO','CO_NOAA_14_rf_dist'          ,'RF_Dist','MSE']
    ,['DE','DE_counties_WGS72'           ,'RF_Dist','NOAA']
    ,['GA','GA_county_split_WGS'         ,'RF_DIST','MSE']
    ,['IA','county_IA_WGS'               ,'RF_Dist','MSE']
    ,['IN','IN_counties_WGS'             ,'RF_Dist','NOAA']
    ,['KY','KY_counties_WGS'             ,'Rf_Dist','NOAA']
    ,['LA','LA_Parish_split_poly'        ,'rf_dist','MSE']
    ,['MD','MD_areas_WGS72'              ,'RF_Dist','NOAA']
    ,['MN','county_MN_WGS'               ,'RF_Dist','MSE']
    ,['NE','county_NE_WGS'               ,'RF_DIST','MSE']
    ,['NJ','NJ_county_WGS72'             ,'Rf_Dist','NOAA']
    ,['NV','NV_County_WGS'               ,'NV_RF_regn','NV']
    ,['OH','OH_counties_WGS72'           ,'RF_Dist','NOAA']
    ,['PA','PA_counties_WGS72'           ,'RF_Dist','NOAA']
    ,['SC','SC_NOAA_14_rf_dist'          ,'Rf_Dist','NOAA']
    ,['TN','TN_NOAA_14_rf_dist'          ,'Rf_Dist','NOAA']
    ,['WV','WV_rainfall_distribution'    ,'rf_dist','NOAA']
    ,['MA','MA_county_WGS72'             ,'Rf_Dist','NRCC']
    ,['ME','ME_counties_WGS72'           ,'Rf_Dist','NRCC']
    ,['NH','NH_towns_WGS'                ,'Rf_Dist','NRCC']
    ,['NY','NY_counties_WGS72'           ,'Rf_Dist','NRCC']
    ,['RI','RI_counties_WGS72'           ,'Rf_Dist','NRCC']
    ,['VT','VT_counties_WGS72'           ,'Rf_Dist','NRCC']
];

for item in layers:
    print("  processing " + item[1]);
    
    if arcpy.Exists(wrkfgdb + os.sep + item[1] + "_NAD83"):
        arcpy.Delete_management(wrkfgdb + os.sep + item[1] + "_NAD83");
        
    if arcpy.Exists(wrkfgdb + os.sep + item[1] + "_RDY"):
        arcpy.Delete_management(wrkfgdb + os.sep + item[1] + "_RDY");
    
    arcpy.management.Project(
         in_dataset      = fgdb    + os.sep + item[1]
        ,out_dataset     = wrkfgdb + os.sep + item[1] + "_NAD83"
        ,out_coor_system = arcpy.SpatialReference(4269)
    );
    
    arcpy.CreateFeatureclass_management(
         out_path      = wrkfgdb
        ,out_name      = item[1] + "_RDY"
        ,geometry_type = "POLYGON"
        ,has_m         = "DISABLED"
        ,has_z         = "DISABLED"
        ,spatial_reference = arcpy.SpatialReference(4326) 
    );

    arcpy.management.AddFields(
         in_table          = wrkfgdb + os.sep + item[1] + "_RDY"
        ,field_description = [
             ['state'     ,'TEXT','state'     ,2]
            ,['rf_value'  ,'TEXT','rf_value'  ,255]
            ,['source_rf' ,'TEXT','source_rf' ,255]
            ,['source_geo','TEXT','source_geo',255]
        ]
    );
    
    fldsout = [
         'state'
        ,'rf_value'
        ,'source_rf'
        ,'source_geo'
        ,'SHAPE@'
    ];
    
    fldsin = [
         item[2]
        ,'SHAPE@'
    ];

    with arcpy.da.InsertCursor(wrkfgdb + os.sep + item[1] + "_RDY",fldsout) as outcur:
        
        with arcpy.da.SearchCursor(wrkfgdb + os.sep + item[1] + "_NAD83",fldsin) as incur:
            
            for row in incur:
                rf_value = str(row[0]);
                
                if item[3] == 'CA':
                    if rf_value == 'CA-1':
                        rf_value = 'CA_1';
                    elif rf_value == 'CA-2':
                        rf_value = 'CA_2';
                    elif rf_value == 'CA-3':
                        rf_value = 'CA_3';
                    elif rf_value == 'CA-4':
                        rf_value = 'CA_4';
                    elif rf_value == 'CA-5':
                        rf_value = 'CA_5';
                    elif rf_value == 'CA-6':
                        rf_value = 'CA_6';
                    else:
                        raise Exception('err');
                
                else:
                    if rf_value == 'C or D':
                        raise Exception('found unusable CT rf values');
                        
                    rf_value = item[3] + '_' + rf_value;
                    
                outcur.insertRow((
                     item[0]
                    ,rf_value
                    ,item[1]
                    ,item[1]
                    ,row[1]
                ));


  processing AL_county_split_rf_dist__WGS
  processing AR_county_split_WGS
  processing CA_6_dist_map
  processing CO_NOAA_14_rf_dist
  processing DE_counties_WGS72
  processing GA_county_split_WGS
  processing county_IA_WGS
  processing IN_counties_WGS
  processing KY_counties_WGS
  processing LA_Parish_split_poly
  processing MD_areas_WGS72
  processing county_MN_WGS
  processing county_NE_WGS
  processing NJ_county_WGS72
  processing NV_County_WGS
  processing OH_counties_WGS72
  processing PA_counties_WGS72
  processing SC_NOAA_14_rf_dist
  processing TN_NOAA_14_rf_dist
  processing WV_rainfall_distribution
  processing MA_county_WGS72
  processing ME_counties_WGS72
  processing NH_towns_WGS
  processing NY_counties_WGS72
  processing RI_counties_WGS72
  processing VT_counties_WGS72
Wall time: 2min 17s


### Create the RainfallDistribution Layer

In [14]:
%%time

wrkprecip = wrkfgdb + os.sep + 'RainfallDistribution';

if arcpy.Exists(wrkprecip):
    arcpy.Delete_management(wrkprecip);
       
print("  building fresh Rainfall Distribution layer")
arcpy.CreateFeatureclass_management(
     out_path      = wrkfgdb
    ,out_name      = 'RainfallDistribution'
    ,geometry_type = "POLYGON"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269)
);

arcpy.management.AddFields(
     in_table          = wrkprecip
    ,field_description = [
         ['state'     ,'TEXT','state'     ,2]
        ,['rf_value'  ,'TEXT','rf_value'  ,255]
        ,['source_rf' ,'TEXT','source_rf' ,255]
        ,['source_geo','TEXT','source_geo',255]
    ]
);

if arcpy.Exists(wrkfgdb + os.sep + 'ireg_zones_NAD83'):
    arcpy.Delete_management(wrkfgdb + os.sep + 'ireg_zones_NAD83');

print("  projecting base ireg_zones to NAD83");
arcpy.management.Project(
     in_dataset      = fgdb + os.sep + 'ireg_zones'
    ,out_dataset     = wrkfgdb + os.sep + 'ireg_zones_NAD83'
    ,out_coor_system = arcpy.SpatialReference(4269)
);

fldsout = [
     'state' 
    ,'rf_value'
    ,'source_rf'
    ,'source_geo'
    ,'SHAPE@'
];

fldsin = [
     'ireg'
    ,'SHAPE@'
]; 

print("  writing out base ireg_zones into Rainfall Distribution layer")
with arcpy.da.InsertCursor(wrkprecip,fldsout) as outcur:
    
    with arcpy.da.SearchCursor(wrkfgdb + os.sep + 'ireg_zones_NAD83',fldsin) as incur:
        for row in incur:
            
            outcur.insertRow((
                 'NA'
                ,'SCS_' + row[0]
                ,'ireg_zones'
                ,'ireg_zones'
                ,row[1]
            ));


  building fresh Rainfall Distribution layer
  projecting base ireg_zones to NAD83
  writing out base ireg_zones into Rainfall Distribution layer
Wall time: 6.1 s


### Merge the various state layers onto the Rainfall Distribution Coverage

In [32]:
%%time

rfcov = rezfgdb + os.sep + 'RainfallDistribution';

if arcpy.Exists(rfcov):
    arcpy.Delete_management(rfcov);
        
tmprf = 'memory' + os.sep + 'RainfallDistribution_tmp';

if arcpy.Exists(tmprf):
    arcpy.Delete_management(tmprf);
    
tmprf2 = 'memory' + os.sep + 'RainfallDistribution_tmp2';

if arcpy.Exists(tmprf2):
    arcpy.Delete_management(tmprf2);
    
tmprf3 = 'memory' + os.sep + 'RainfallDistribution_tmp3';
tmprf3 = wrkfgdb + os.sep + 'RainfallDistribution_tmp3';

if arcpy.Exists(tmprf3):
    arcpy.Delete_management(tmprf3);

layers = [
     'AL_county_split_rf_dist__WGS_RDY'
    ,'AR_county_split_WGS_RDY'
    ,'CA_6_dist_map_RDY'
    ,'CO_NOAA_14_rf_dist_RDY'
    ,'DE_counties_WGS72_RDY'
    ,'GA_county_split_WGS_RDY'
    ,'county_IA_WGS_RDY'
    ,'IN_counties_WGS_RDY'
    ,'KY_counties_WGS_RDY'
    ,'LA_Parish_split_poly_RDY'
    ,'MD_areas_WGS72_RDY'
    ,'county_MN_WGS_RDY'
    ,'county_NE_WGS_RDY'
    ,'NJ_county_WGS72_RDY'
    ,'NV_County_WGS_RDY'
    ,'OH_counties_WGS72_RDY'
    ,'PA_counties_WGS72_RDY'
    ,'SC_NOAA_14_rf_dist_RDY'
    ,'TN_NOAA_14_rf_dist_RDY'
    ,'VA_counties_and_zones_RDY'
    ,'WV_rainfall_distribution_RDY'
    ,'CT_counties_WGS72_RDY'
    ,'MA_county_WGS72_RDY'
    ,'ME_counties_WGS72_RDY'
    ,'NH_towns_WGS_RDY'
    ,'NY_counties_WGS72_RDY'
    ,'RI_counties_WGS72_RDY'
    ,'VT_counties_WGS72_RDY'
    ,'county_FL_WGS_split_poly9_RDY'
    ,'rainfall_IL_RDY'
    ,'rainfall_NC_RDY'
    ,'rainfall_AK_RDY'
];

infc   = None;
outfc  = None;
swapfc = None;

if arcpy.CheckProduct("ArcInfo") == "Available":
    print("  using Advanced license features...");
else:
    print("  using Basic license features...");
        
for i,layer in enumerate(layers):
    print("  processing " + layer);
    
    if infc is None:
        infc   = wrkprecip;
        outfc  = tmprf;
        swapfc = tmprf2;

    elif i == len(layers) - 1:
        infc   = outfc;
        outfc  = rezfgdb + os.sep + 'RainfallDistribution';
        swapfc = outfc;

    else:
        infc   = outfc;
        outfc  = swapfc;
        swapfc = infc;

    if arcpy.Exists(outfc):
        arcpy.Delete_management(outfc);
        
    if arcpy.CheckProduct("ArcInfo") == "Available":
        
        arcpy.analysis.Update(
             in_features       = infc
            ,update_features   = wrkfgdb + os.sep + layer
            ,out_feature_class = outfc
            ,keep_borders      = 'BORDERS'
        );       

    else:
        
        if arcpy.Exists(tmprf3):
            arcpy.Delete_management(tmprf3);
        
        arcpy.analysis.Union(
             in_features       = [
                  infc
                 ,wrkfgdb + os.sep + layer
             ]
            ,out_feature_class = tmprf3
        );
        
        with arcpy.da.UpdateCursor(
             in_table     = tmprf3
            ,field_names  = ['OBJECTID']
            ,where_clause = "FID_" + os.path.basename(infc) + " = -1 OR FID_" + layer + " <> -1"
        ) as upcur:
            for row in upcur:
                upcur.deleteRow();

        arcpy.management.DeleteField(
             in_table     = tmprf3
            ,drop_field   = [
                 'FID_' + os.path.basename(infc)
                ,'FID_' + layer
                ,'state_1'
                ,'rf_value_1'
                ,'source_rf_1'
                ,'source_geo_1'
            ]
        );
        
        arcpy.Append_management(
             inputs      = wrkfgdb + os.sep + layer
            ,target      = tmprf3
        );
        
        arcpy.conversion.FeatureClassToFeatureClass(
             in_features = tmprf3
            ,out_path    = os.path.dirname(outfc)
            ,out_name    = os.path.basename(outfc)
        );
            

  using Advanced license features...
  processing AL_county_split_rf_dist__WGS_RDY
  processing AR_county_split_WGS_RDY
  processing CA_6_dist_map_RDY
  processing CO_NOAA_14_rf_dist_RDY
  processing DE_counties_WGS72_RDY
  processing GA_county_split_WGS_RDY
  processing county_IA_WGS_RDY
  processing IN_counties_WGS_RDY
  processing KY_counties_WGS_RDY
  processing LA_Parish_split_poly_RDY
  processing MD_areas_WGS72_RDY
  processing county_MN_WGS_RDY
  processing county_NE_WGS_RDY
  processing NJ_county_WGS72_RDY
  processing NV_County_WGS_RDY
  processing OH_counties_WGS72_RDY
  processing PA_counties_WGS72_RDY
  processing SC_NOAA_14_rf_dist_RDY
  processing TN_NOAA_14_rf_dist_RDY
  processing VA_counties_and_zones_RDY
  processing WV_rainfall_distribution_RDY
  processing CT_counties_WGS72_RDY
  processing MA_county_WGS72_RDY
  processing ME_counties_WGS72_RDY
  processing NH_towns_WGS_RDY
  processing NY_counties_WGS72_RDY
  processing RI_counties_WGS72_RDY
  processing VT_counti

In [16]:
rfcov_cnt = arcpy.GetCount_management(rfcov)[0];
                            
print("Rainfall Distribution Coverage Count: " + str(rfcov_cnt));                      
                            

Rainfall Distribution Coverage Count: 19613
